# 第 14 章　可迭代的对象、迭代器和生成器

迭代是数据处理的基石。扫描内存中放不下的数据集时，我们要找到一
种惰性获取数据项的方式，即按需一次获取一个数据项。这就是迭代器
模式（Iterator pattern）。本章说明 Python 语言是如何内置迭代器模式
的，这样就避免了自己手动去实现。

与 Lisp不同，Python 没有宏，因此为了抽象出迭代器模式，需要改动语言本身。
为此，Python 2.2（2001 年）加入了 yield 关键字。
这个关键字用于构建生成器（generator），其作用与迭代器一样。

所有生成器都是迭代器，因为生成器完全实现了迭代器接口。
但，**迭代器**用于从集合中**抽取**元素；而**生成器**用于”凭空“**生成**元素。
python社区不区分两者区别。

在 Python 中，所有集合都可以迭代。在 Python 语言内部，迭代器用于支持：
- for 循环
- 构建和扩展集合类型
- 逐行遍历文本文件
- 列表推导、字典推导和集合推导
- 元组拆包
- 调用函数时，使用 * 拆包实参

本章内容：
- 语言内部使用 iter(...) 内置函数处理可迭代对象的方式
- 如何使用 Python 实现经典的迭代器模式
- 详细说明生成器函数的工作原理
- 如何使用生成器函数或生成器表达式代替经典的迭代器
- 如何使用标准库中通用的生成器函数
- 如何使用 yield from 语句合并生成器
- 案例分析：在一个数据库转换工具中使用生成器函数处理大型数据集
- 为什么生成器和协程看似相同，实则差别很大，不能混淆

## 14.1 单词序列案例

In [ ]:
from sentence import Sentence

s = Sentence('"The time has come," the Walrus said,')
s

In [3]:
for w in s:
    print(w)

The
time
has
come
the
Walrus
said


In [4]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [5]:
s[3]

'come'

### 序列可以迭代的原因：iter函数

解释器需要迭代对象 x 时，会自动调用 iter(x)。
内置的 iter 函数有以下作用。
- (1) 检查对象是否实现了 __iter__ 方法，如果实现了就调用它，获取
一个迭代器。
- (2) 如果没有实现 __iter__ 方法，但是实现了 __getitem__ 方法，
Python 会创建一个迭代器，尝试按顺序（从索引 0 开始）获取元素。
- (3) 如果尝试失败，Python 抛出 TypeError 异常，通常会提示“C object
is not iterable”（C 对象不可迭代），其中 C 是目标对象所属的类。

In [1]:
# 判断一个类是否是可以迭代的，建议使用下面的方法，而不要使用issubclass(abc.Iterable)
# 因为iter()只要实现__getitem__或__iter__其中一个就可以，而使用subclass检查必须实现__iter__
iter(s)

NameError: name 's' is not defined

- 如果对象实现了能返回迭代器的 __iter__ 方法，那么对象就是可迭代的。序列都可以迭代；
- 实现了 __getitem__ 方法，而且其参数是从零开始的索引，这种对象也可以迭代。

我们要明确可迭代的对象和迭代器之间的关系：Python 从可迭代的对象中获取迭代器。

In [10]:
s = 'ABC'
i = iter(s)
while True:
    try:
        print(next(i))
    except StopIteration:
        del i
        break

A
B
C


## 　Sentence类第2版：典型的迭代器

因此，迭代器可以迭代，但是可迭代的对象不是迭代器。

除了 \_\_iter__ 方法之外，你可能还想在 Sentence 类中实现
\_\_next__ 方法，让 Sentence 实例既是可迭代的对象，也是自身的
迭代器。可是，这种想法非常糟糕。根据有大量 Python 代码审查经验的
Alex Martelli 所说，这也是常见的反模式。

迭代器模式可用来：
- 访问一个聚合对象的内容而无需暴露它的内部表示
- 支持对聚合对象的多种遍历
- 为遍历不同的聚合结构提供一个统一的接口（即支持多态迭代）

可迭代的对象一定不能是自身的迭代器。也就是说，可迭代的对象
必须实现 \_\_iter__ 方法，但不能实现 \_\_next__ 方法。

In [ ]:
import re
import reprlib

"""
第二版
实现迭代器
"""

RE_WORD = re.compile('\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)

    def __iter__(self):
        return SentenceIterator(self.words)


class SentenceIterator:
"""
    Sentence类的迭代器
"""
    def __init__(self, words):
        self.words = words
        self.index = 0

    # 返回下一个可用的元素，如果没有元素了，抛出 StopIteration异常。
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word

    # 返回 self，以便在应该使用可迭代对象的地方使用迭代器，例如在 for 循环中。
    def __iter__(self):
        return self



"""
因此，迭代器可以迭代，但是可迭代的对象不是迭代器。

除了 __iter__ 方法之外，你可能还想在 Sentence 类中实现
__next__ 方法，让 Sentence 实例既是可迭代的对象，也是自身的
迭代器。可是，这种想法非常糟糕。根据有大量 Python 代码审查经验的
Alex Martelli 所说，这也是常见的反模式。



迭代器模式可用来：
- 访问一个聚合对象的内容而无需暴露它的内部表示
- 支持对聚合对象的多种遍历
- 为遍历不同的聚合结构提供一个统一的接口（即支持多态迭代）
"""

## 　Sentence类第3版：生成器函数

In [ ]:
import re
import reprlib

"""
第三版
符合Python习惯的实现迭代器
"""

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)

    def __iter__(self):
        for word in self.words:
            yield word
        return  # 这个 return 语句不是必要的；这个函数可以直接“落空”，自动返
                # 回。不管有没有 return 语句，生成器函数都不会抛出
                # StopIteration 异常，而是在生成完全部值之后会直接退出。

    """
    # 也可使用更加简洁的方法：
    def __iter__(self):
        return iter(self.words) # 调用 __iter__ 方法得到的就是迭代器
    """

只要 Python 函数的定义体中有 yield 关键字，该函数就是生成器函
数。调用生成器函数时，会返回一个生成器对象。也就是说，生成器函
数是生成器工厂。

In [11]:
# 理解yield关键字

def gen_123(): #只要 Python 函数中包含关键字 yield，该函数就是生成器函数
    yield 1
    yield 2
    yield 3

gen_123
# <function __main__.gen_123()>

<function __main__.gen_123()>

In [12]:
gen_123()
# <generator object gen_123 at 0x01467108>

<generator object gen_123 at 0x01467108>

In [13]:
for i in gen_123():
    print(i)


1
2
3


In [14]:
g = gen_123()
next(g)

1

In [15]:
next(g)


2

In [16]:
next(g)

3

In [20]:
next(g)
# StopIteration

StopIteration: 

生成器函数会创建一个生成器对象，包装生成器函数的定义体。把生成
器传给 next(...) 函数时，生成器函数会向前，执行函数定义体中的
下一个 yield 语句，返回产出的值，并在函数定义体的当前位置暂
停。最终，函数的定义体返回时，外层的生成器对象会抛出
StopIteration 异常——这一点与迭代器协议一致。

In [21]:
# 生成器函数定义体的执行过程：
def gen_AB():
    print('start')
    yield "A"
    print('continue')
    yield 'B'
    print('end')

for c in gen_AB():
    print('-->', c)

start
--> A
continue
--> B
end


In [26]:
next(gen_AB())

start


'A'

## Sentence类第4版：惰性实现

懒惰的反义词是急迫，其实，惰性求值（lazy evaluation）
和及早求值（eager evaluation）是编程语言理论方面的技术术语。
目前实现的几版 Sentence 类都不具有惰性，因为 \_\_init__ 方法急
迫地构建好了文本中的单词列表，然后将其绑定到 self.words 属性
上。这样就得处理整个文本，列表使用的内存量可能与文本本身一样多
（或许更多，这取决于文本中有多少非单词字符）。如果只需迭代前几
个单词，大多数工作都是白费力气。

In [ ]:
import re
import reprlib

"""
第四版
惰性实现
"""

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)

    """
    def __iter__(self):
        for word in self.words:
            yield word
        return
    """
    def __iter__(self):
        for match in RE_WORD.finditer(self.text): # finditer 函数构建一个迭代器，包含 self.text 中匹配
                                                # RE_WORD 的单词，产出 MatchObject 实例。
            yield match.group()  # match.group() 方法从 MatchObject 实例中提取匹配正则表达
                                 # 式的具体文本。

## Sentence类第5版：生成器表达式
生成器表达式可以理解为列表推导的惰性版本：不会迫切地构建列表，
而是返回一个生成器，按需惰性生成元素。也就是说，如果列表推导是
制造列表的工厂，那么生成器表达式就是制造生成器的工厂。

In [38]:
# 在列表推导中使用 gen_AB 生成器函数
def gen_AB():
    print('start')
    yield "A"
    print('continue')
    yield 'B'
    print('end')

res1 = [x*3 for x in gen_AB()] # 列表推导
res1

start
continue
end


['AAA', 'BBB']

In [39]:
for x in res1:
    print("-->>", x)

-->> AAA
-->> BBB


In [42]:
res2 = (x*3 for x in gen_AB()) # 生成器表达式
res2

<generator object <genexpr> at 0x0148D108>

In [43]:
for x in res2:
    print("-->>", x)

start
-->> AAA
continue
-->> BBB
end


res2 是一个生成器对象。

只有 for 循环迭代 res2 时，gen_AB 函数的定义体才会真正执
行。for 循环每次迭代时会隐式调用 next(res2)，前进到 gen_AB
函数中的下一个 yield 语句。注意，gen_AB 函数的输出与 for 循环
中 print 函数的输出夹杂在一起。

In [ ]:
import re
import reprlib

"""
第四版
惰性实现
"""
RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)
    """ 第一版
    def __getitem__(self, item):
        return self.words[item]

    def __len__(self):
        return len(self.words)
    """
    """第二版
    def __iter__(self):
        return SentenceIterator(self.words)
    """

    """第三版
    def __iter__(self):
        for word in self.words:
            yield word
        return
    """
    """第四版
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()
    """
    # 第五版 使用生成器表达式惰性生成迭代器！
    def __iter__(self):
        return (yield match.group() for match in RE_WORD.finditer(self.text))
